In [34]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from joblib import Parallel, delayed


In [35]:
r = requests.get('https://myanimelist.net/anime/genre/1/Action?page=1')
soup = BeautifulSoup(r.text, 'html.parser')

In [36]:
soup.find('h3',class_='h3_anime_subtitle')

In [37]:
soup.find_all('a',class_='link-title')

[]

In [38]:
product_links = set()

for x in range(1, 5):
    r = requests.get(f'https://myanimelist.net/anime/genre/1/Action?page={x}')
    soup = BeautifulSoup(r.text, 'html.parser')
    anime_links = soup.find_all('a', class_='link-title')

    for link in anime_links:
        title = link.text
        url = link['href']
        product_links.add((title, url))
        print(f"Title: {title}\nLink: {url}\n")

product_links = list(product_links)

In [39]:
r = requests.get('https://myanimelist.net/anime/11757/Sword_Art_Online')
soup = BeautifulSoup(r.text, 'html.parser')

In [40]:
soup.find_all('span',class_='dark_text')

[]

In [41]:
r = requests.get('https://myanimelist.net/anime/11757/Sword_Art_Online')
soup = BeautifulSoup(r.text, 'html.parser')

divs = soup.find_all('div', class_='spaceit_pad')
anime_info = {}

for div in divs:
    span = div.find('span', class_='dark_text')
    if span:
        label = span.get_text(strip=True)
        value = div.get_text(strip=True, separator=' ')[len(label) + 1:].strip()
        # Fix Genres and Themes
        if label == 'Genres:':
            value = [genre.strip() for genre in value.split(',')]
        elif label == 'Themes:':
            value = [theme.strip() for theme in value.split(',')]
        anime_info[label] = value

for key, value in anime_info.items():
    print(f"{key}: {', '.join(value) if isinstance(value, list) else value}")



In [42]:
def extract_anime_info(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')

    divs = soup.find_all('div', class_='spaceit_pad')
    anime_info = {}

    for div in divs:
        span = div.find('span', class_='dark_text')
        if span:
            label = span.get_text(strip=True)
            value = div.get_text(strip=True, separator=' ')[len(label) + 1:].strip()
            # Fix Genres and Themes
            if label == 'Genres:':
                value = [genre.strip() for genre in value.split(',')]
            elif label == 'Themes:':
                value = [theme.strip() for theme in value.split(',')]
            anime_info[label] = value

    return anime_info

product_links = set()

for x in range(1, 2):
    r = requests.get(f'https://myanimelist.net/anime/genre/1/Action?page={x}')
    soup = BeautifulSoup(r.text, 'html.parser')
    anime_links = soup.find_all('a', class_='link-title')

    for link in anime_links:
        title = link.text
        url = link['href']
        product_links.add((title, url))

product_links = list(product_links)

for title, url in product_links:
    anime_info = extract_anime_info(url)
    print(f"Title: {title}")
    for key, value in anime_info.items():
        print(f"{key}: {', '.join(value) if isinstance(value, list) else value}")
    print()


In [43]:
def extract_anime_info(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')

    divs = soup.find_all('div', class_='spaceit_pad')
    anime_info = {}

    for div in divs:
        span = div.find('span', class_='dark_text')
        if span:
            label = span.get_text(strip=True)
            value = div.get_text(strip=True, separator=' ')[len(label) + 1:].strip()
            # Fix Genres and Themes
            if label == 'Genres:':
                value = ', '.join([genre.strip() for genre in value.split(',')])
            elif label == 'Themes:':
                value = ', '.join([theme.strip() for theme in value.split(',')])
            anime_info[label[:-1]] = value  # Remove the trailing colon

    return anime_info

product_links = []

for x in range(1, 2):  #Modify the number of pages here 
    r = requests.get(f'https://myanimelist.net/anime/genre/1/Action?page={x}')
    soup = BeautifulSoup(r.text, 'html.parser')
    anime_links = soup.find_all('a', class_='link-title')

    for link in anime_links:
        title = link.text
        url = link['href']
        product_links.append((title, url))

anime_data = []

for title, url in product_links:
    anime_info = extract_anime_info(url)
    anime_info['Title'] = title
    anime_data.append(anime_info)

df = pd.DataFrame(anime_data)

columns_order = ['Title', 'Synonyms', 'Japanese', 'English', 'Type', 'Episodes', 'Status', 'Aired', 
                 'Premiered', 'Broadcast', 'Producers', 'Licensors', 'Studios', 'Source', 'Genres', 
                 'Themes', 'Demographic', 'Duration', 'Rating', 'Score', 'Ranked', 'Popularity', 
                 'Members', 'Favorites']

df = df.reindex(columns=columns_order)

In [ ]:
df.to_csv('anime.csv')

Adventure

In [ ]:
def extract_anime_info(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')

    divs = soup.find_all('div', class_='spaceit_pad')
    anime_info = {}

    for div in divs:
        span = div.find('span', class_='dark_text')
        if span:
            label = span.get_text(strip=True)
            value = div.get_text(strip=True, separator=' ')[len(label) + 1:].strip()
            if label == 'Genres:':
                value = ', '.join([genre.strip() for genre in value.split(',')])
            elif label == 'Themes:':
                value = ', '.join([theme.strip() for theme in value.split(',')])
            anime_info[label[:-1]] = value  

    return anime_info

product_links = []

for x in range(1, 2):  #Modify the number of pages here 
    r = requests.get(f'https://myanimelist.net/anime/genre/2/Adventure?page={x}')  #follow the similar approach for all the Genres
    soup = BeautifulSoup(r.text, 'html.parser')
    anime_links = soup.find_all('a', class_='link-title')

    for link in anime_links:
        title = link.text
        url = link['href']
        product_links.append((title, url))
        
anime_data = []

for title, url in product_links:
    anime_info = extract_anime_info(url)
    anime_info['Title'] = title
    anime_data.append(anime_info)

df = pd.DataFrame(anime_data)

columns_order = ['Title', 'Synonyms', 'Japanese', 'English', 'Type', 'Episodes', 'Status', 'Aired', 
                 'Premiered', 'Broadcast', 'Producers', 'Licensors', 'Studios', 'Source', 'Genres', 
                 'Themes', 'Demographic', 'Duration', 'Rating', 'Score', 'Ranked', 'Popularity', 
                 'Members', 'Favorites']

df = df.reindex(columns=columns_order)

In [ ]:
df.to_csv('anime_adventure.csv')